# Predicting Q3 Protein Secondary Structure Using a Traditional HMM
### COMS 4761 - Project



### Step 0: Create All Functions that Will Be Used Downstream

In [ ]:
import gzip
import numpy as np
import pandas as pd
import pickle
import time
from sklearn.model_selection import train_test_split
import random

In [ ]:
def get_data(arr, residue_list, q8_list, columns, r, f, bounds=None):
    
    """
    This function retrieves and formats data from the CB6133_filtered and CB531 datasets [1][2]
    Codes is slighlty modified from code provided by [3][4]
    
    [1] Jian Zhou and Olga G. Troyanskaya. Deep supervised and convolutional generative stochastic network for
        protein s
    [2] Jian Zhou and Olga G. Troyanskaya. CB6133 dataset.
        https://www.princeton.edu/~jzthree/datasets/ICML2014/dataset_readme.txt, 2014.
    [3] Iddo Drori et al. High Quality Prediction of Protein Q8 Secondary Structure by
        Diverse Neural Network Architectures. arXiv preprint arXiv:1811.07143, 2018
    [4] https://github.com/idrori/cu-ssp/blob/master/model_1/model_1.py
    """
    
    if bounds is None: bounds = range(len(arr))
    
    data = [None for i in bounds]
    for i in bounds:
        seq, q8, q3, q2, profiles = '', '', '', '', []
        for j in range(r):
            jf = j*f
            
            # Residue convert from one-hot to decoded
            residue_onehot = arr[i,jf+0:jf+22]
            residue = residue_list[np.argmax(residue_onehot)]

            # Q8 one-hot encoded to decoded structure symbol
            residue_q8_onehot = arr[i,jf+22:jf+31]
            residue_q8 = q8_list[np.argmax(residue_q8_onehot)]

            if residue == 'NoSeq': break      # terminating sequence symbol

            nc_terminals = arr[i,jf+31:jf+33] # nc_terminals = [0. 0.]
            sa = arr[i,jf+33:jf+35]           # sa = [0. 0.]
            profile = arr[i,jf+35:jf+57]      # profile features
            
            seq += residue # concat residues into amino acid sequence
            #encode q3 structure
            if residue_q8 in 'GHI':
                q3 += 'H'
                q2 += 'A'
            elif residue_q8 in 'E':
                q3 += 'E'
                q2 += 'X'
            else:
                q3 += 'C'
                q2 += 'X'
            
            q8  += residue_q8 # concat secondary structure into secondary structure sequence
            profiles.append(profile)
        
        data[i] = [str(i+1), len(seq), seq, np.array(profiles), q8, q3, q2]
    
    return pd.DataFrame(data, columns=columns)


def encode_sequence(sequence, code):
    
    """
    Provided an input sequence and a code, returns the encoding of the sequence
    """
    
    encoded_seq = []
    
    for x in sequence:
        try:
            idx = code[x]
            encoded_seq.append(idx)
        except Exception as e:
            print(f"Error: {e}")
            break
    
    return encoded_seq


def format_dataset(df, emission_code, state_code, exp_col="q3_expected"):
    
    """
    Provided a dataframe which contains the amino sequences and the hidden sequence,
    this function encodes those sequences according to the provided codes
    and return them
    
    *exp_col specifies if want to encode the q8, q3, or q2 hidden sequence
    """
    
    assert ('id' in df.columns and 'len' in df.columns and 'input' in df.columns and exp_col in df.columns)
    
    formattedDF = pd.DataFrame(columns=['id','len','input','expected'])

    for i in range(len(df)):
        
        sid = df.iloc[i].id
        slen = df.iloc[i].len
        enc_input = encode_sequence(df.iloc[i].input, emission_code)
        enc_expected = encode_sequence(df.iloc[i][exp_col], state_code)
        
        assert (len(enc_input) == len(enc_expected))
        
        formattedDF = formattedDF.append({'id':sid, 'len':slen, 'input':enc_input, 'expected':enc_expected}, ignore_index=True)

    return formattedDF

def estimate_transition_matrix(df, state_code):
    """
    Given a dataframe that has the data for the amino sequences and their corresponding hidden sequence,
    we use the data to compute the MLEs of the emission probablities.
    
    ex. estimated P(emission=A|state=H) = count(emission=A,state=H) / sum_over_all_emission count(emission, state=H)
    
    *implemented a pseudocount of +1 for cases where we have 0 observations of a certain (emission,state) combo
    """
    
    n_states = len(state_code)
    
    #using pseudocount of +1
    counts = np.ones(shape=(n_states, n_states), dtype=float)
    
    for i in range(len(df)):
        
        state_seq = df.iloc[i].expected
        seq_len = df.iloc[i].len
        
        for j in range(seq_len - 1):
            
            x = state_seq[j]
            y = state_seq[j+1]
            
            counts[x,y] += 1
    
    #transform counts to probability by normalizing of row sums
    row_sums = np.sum(counts, axis=1)
    T = counts / row_sums.reshape((-1,1))
    
    return T


def estimate_emission_matrix(df, state_code, emission_code):
    """
    Given a dataframe that has the data for the amino sequences and their corresponding hidden sequence,
    we use the data to compute the MLEs of the transition probablities.
    
    ex. estimated P(state_{t+1}=E|state_{t}=H) = count(state_{t}=H, state_{t+1}=E) / sum_over_all_states count(state_{t}=H, state_{t+1})
    
    *implemented a pseudocount of +1 for cases where we have 0 observations of a certain (state,state) combo
    """
    
    n_states = len(state_code)
    n_emissions = len(emission_code)
    
    #using pseudocount of +1
    counts = np.ones(shape=(n_states, n_emissions), dtype=float)
    
    for i in range(len(df)):
        
        state_seq = df.iloc[i].expected
        emission_seq = df.iloc[i].input
        seq_len = df.iloc[i].len
        
        for j in range(seq_len):
            
            x = state_seq[j]
            y = emission_seq[j]
            
            counts[x,y] += 1

    #transform counts to probability by normalizing of row sums
    row_sums = np.sum(counts, axis=1)
    E = counts / row_sums.reshape((-1,1))
    
    return E

def start_distribution(df,state_code):
    """
    Given a dataframe that has the data for the amino sequences and their corresponding hidden sequence,
    we use the data to compute the MLEs of the start distribution.
    
    ex. estimated P(state=H) = count(state=H) / sum_over_all_states count(state)
    
    *implemented a pseudocount of +1 for cases where we have 0 observations of a certain state
    """
    
    n_states = len(state_code)
    
    #using pseudocount of +1
    counts = np.array([1.0] * n_states)
    
    for i in range(len(df)):
        
        state_seq = df.iloc[i].expected
        seq_len = df.iloc[i].len
        
        for j in range(seq_len):
            
            x = state_seq[j]
            
            counts[x] += 1
    
    #transform counts to probability by normalizing of row sums
    total = sum(counts)
    pi = counts / total
    
    return pi

def viterbi_decoding(T,E,pi,seq):
    """
    This functions performs viterbi decoding to get the predicted hidden sequence,
    given the input emission sequence as well as the transition matrix, emission matrix,
    and the start distribution
    """
    
    #sequence length
    N = len(seq)
    #num of states
    M = T.shape[0]
    
    assert (M == len(pi))
    
    #V will store viterbi values
    V = np.zeros(shape=(M, N), dtype=float)
    
    #P will store prev state from which we transitioned into state m and time n to achieve the max value of V[m,n]
    #(i.e. pointer to help us reconstruct sequence after predicting most probable path in viterbi graph)
    P = np.empty(shape=(M, N))
    P[:] = np.NaN
    
    #populate viterbi matrix
    for n in range(N):
        
        #get current emissions
        e = seq[n]
        
        for m in range(M):
            
            #initilize viterbi value for current timestep given state m to be -infty
            maxV = float("-inf")
            prev = np.NaN
            
            #get log prob. of emission given state m
            emiss_logp = np.log(E[m,e])
            
            #start of sequence
            if n == 0:
                start_logp = np.log(pi[m])
                maxV = emiss_logp + start_logp
                
            else:
            
                #solve for max value for V[m,n]
                for i in range(M): 

                    #get previous timestep viterbi value for state i (which should be a log prob)
                    prev_vit = V[i, n-1]

                    #get log prob of transition from state i to m
                    trans_logp = np.log(T[i,m])

                    #update viterbi value for current timestep given state m
                    curV = prev_vit + trans_logp + emiss_logp

                    if curV > maxV:
                        maxV = curV
                        prev = i
        
            V[m,n] = maxV
            P[m,n] = prev
    
    #initialize with state with highest probability at end of sequence
    best_path = [np.argmax(V[:,-1])]
    
    #work backwards to reconstruct sequence
    for n in range(N-1,0,-1):
        
        #determine where we are
        cur_state = best_path[0]

        #find state from which we came that yielded highest probability to current state at current time
        prev_state = int(P[cur_state,n])

        #prepend previous state
        best_path = [prev_state] + best_path
    
    return best_path


def getPredictions(df, T, E, pi):
    
    """
    get predictions for all sequences in specified dataset using provided HMM
    """
    
    results = pd.DataFrame(columns=['input','predicted','expected'])
    
    for i in range(len(df)):
        
        amino_seq = df.iloc[i].input
        pred_seq = viterbi_decoding(T,E,pi,amino_seq)
        exp_seq = df.iloc[i].expected
        
        assert(len(amino_seq) == len(pred_seq) and len(amino_seq) == len(exp_seq))
        
        results = results.append({'input':amino_seq, 'predicted':pred_seq, 'expected':exp_seq}, ignore_index=True)
    
    return results

def HMMaccuracy(df,q=3):
    
    """
    Compute accurcay of HMM given a dataframe the has the input emission sequences,
    the predicted hidden sequences, and the actual hidden sequences
    
    *q specifies whether the predicion was made for q2, q3, or q8 protein structure
    """
    
    #row represents expected state
    #col represents predicted state
    counts = np.zeros(shape=(q,q), dtype=int)
    
    for i in range(len(df)):

        #get predicted and expected hidden sequence from dataframe
        pred = df.iloc[i].predicted
        exp = df.iloc[i].expected
        
        assert (len(pred) == len(exp))
        
        for j in range(len(pred)):
            
            x = exp[j]
            y = pred[j]
            counts[x,y] += 1
    
    rowSum = np.sum(counts, axis=1)
    colSum = np.sum(counts, axis=0)
    
    #true positive (negative) / total predicted positive (negative)
    precision = np.array([counts[i,i] / colSum[i] for i in range(q)])
    
    #true positive (negative) / total actual positive (negative)
    recall = np.array([counts[i,i] / rowSum[i] for i in range(q)])
    
    accuracy = 0
    for i in range(q):
        accuracy += counts[i,i]
    accuracy = accuracy / sum(rowSum)
    
    
    return accuracy, precision, recall, counts
       


### Step 1: Load data

In [ ]:
#seed so get consistent results for every run
random.seed(0)

cb513 = np.load('cb513+profile_split1.npy.gz')
cb6133filtered = np.load('cullpdb+profile_5926_filtered.npy.gz')
print("Data Loaded")
print(f"CB6133 shape: {cb6133filtered.shape}")
print(f"CB513 shape: {cb513.shape}")

Data Loaded
CB6133 shape: (5365, 39900)
CB513 shape: (514, 39900)


### Step 2: Process Data

We split the dataset into train, dev, and test sets.

In [ ]:
maxlen_seq = r = 700 # protein residues padded to 700
f = 57  # number of features for each residue
residue_list = list('ACEDGFIHKMLNQPSRTWVYX') + ['NoSeq']
q8_list      = list('LBEGIHST') + ['NoSeq']
q3_list      = list('HCE') + ['NoSeq']
q2_list      = list('AX') + ['NoSeq']

columns = ["id", "len", "input", "profiles", "q8_expected", "q3_expected", "q2_expected"]

print("Turning data arrays into dataframes")

# train, dev, test split
# break out 10% of train data to be used as dev set
train_df, dev_df = train_test_split(get_data(cb6133filtered, residue_list, q8_list, columns, r, f), test_size=0.1, random_state=11)
test_df  = get_data(cb513, residue_list, q8_list, columns, r, f)

Turning data arrays into dataframes


### Step 3: Encode Sequences and Format DataFrames
    (a) Create codes to encode emission and hidden sequences
    (b) Apply encodings & specify hidden sequence of interest (q2, q3, q8)

In [ ]:
emission_code = {residue_list[i]:i for i in range(len(residue_list)-1)}
state_code = {q3_list[i]:i for i in range(len(q3_list)-1)}

print("emission_code:")
for k,v in emission_code.items():
    print(f"{k}:{v}", end=" ")

print("\n\nstate_code:")
for k,v in state_code.items():
    print(f"{k}:{v}", end=" ")

emission_code:
A:0 C:1 E:2 D:3 G:4 F:5 I:6 H:7 K:8 M:9 L:10 N:11 Q:12 P:13 S:14 R:15 T:16 W:17 V:18 Y:19 X:20 

state_code:
H:0 C:1 E:2 

In [ ]:
print("Encoding sequences")
train_df_formatted = format_dataset(train_df, emission_code, state_code, 'q3_expected')
dev_df_formatted = format_dataset(dev_df, emission_code, state_code, 'q3_expected')
test_df_formatted = format_dataset(test_df, emission_code, state_code, 'q3_expected')    

Encoding sequences


### Step 4: Estimate HMM=(T, E, pi) using Our Data

In [ ]:
print("Computing initial estimates for transition and emission matrices using training data")
start = time.time()
T = estimate_transition_matrix(train_df_formatted, state_code)
E = estimate_emission_matrix(train_df_formatted, state_code, emission_code)
pi = start_distribution(train_df_formatted,state_code)
end = time.time()
print(f"Time to estimate T, E, pi is approx: {round((end-start)//60,4)} minutes")

Computing initial estimates for transition and emission matrices using training data
Time to estimate T, E, pi is approx: 0.0 minutes


### Step 5: Compare HMM against Prior Research [5]

    Some slight difference is expected because they were only able to train and test on CB531 whereas we will be training on CB6113 and testing on CB531.

    [5] W. Ding, D. Dai, J. Xie, H. Zhang, W. Zhang and H. Xie, "PRT-HMM: A Novel Hidden Markov Model for Protein Secondary Structure Prediction," 2012 IEEE/ACIS 11th International Conference on Computer and Information Science, 2012, pp. 207-212, doi: 10.1109/ICIS.2012.89.

    https://ieeexplore-ieee-org.ezproxy.cul.columbia.edu/stamp/stamp.jsp?tp=&arnumber=6211098


In [ ]:
print("Start Distribution (H,C,E):")
print(pi.round(decimals=4))

Start Distribution (H,C,E):
[0.3863 0.3968 0.2169]


In [ ]:
#compare start distribution to source
pi_source = np.array([0.3496, 0.4405, 0.2100] )
pi_delta = (pi - pi_source).round(decimals=4)
pi_delta

array([ 0.0367, -0.0437,  0.0069])

In [ ]:
print("Transition Matrix (H,C,E) x (H,C,E):")
print(T.round(decimals=4))

Transition Matrix (H,C,E) x (H,C,E):
[[0.8989 0.0982 0.0028]
 [0.0944 0.8081 0.0975]
 [0.0093 0.172  0.8186]]


In [ ]:
#compare transition matrix to source
T_source = np.array( \
    [[0.8937, 0.1036, 0.0027], \
     [0.0810, 0.8297, 0.0893], \
     [0.0091, 0.1801, 0.8108 ]]
    )

T_delta = (T - T_source).round(decimals=4)
T_delta

array([[ 0.0052, -0.0054,  0.0001],
       [ 0.0134, -0.0216,  0.0082],
       [ 0.0002, -0.0081,  0.0078]])

In [ ]:
print("Emissions Matrix (H,C,E) x Amino Acids:")
print(E.round(decimals=4))

Emissions Matrix (H,C,E) x Amino Acids:
[[0.1138 0.0103 0.0929 0.0522 0.0357 0.0406 0.0578 0.0208 0.0655 0.0192
  0.1193 0.0328 0.0478 0.0244 0.0485 0.0596 0.0414 0.0149 0.0602 0.035
  0.0074]
 [0.0641 0.012  0.0569 0.0814 0.1195 0.032  0.035  0.026  0.0559 0.013
  0.0633 0.0617 0.0324 0.0796 0.0736 0.0445 0.0596 0.0109 0.0452 0.0282
  0.0054]
 [0.0636 0.016  0.0478 0.0329 0.0475 0.058  0.1008 0.0232 0.0454 0.0155
  0.1035 0.0259 0.0282 0.0191 0.0511 0.0462 0.0657 0.0182 0.1339 0.0503
  0.0072]]


In [ ]:
#compare emission matrix to source
E_source = np.array( \
    [[0.1218, 0.0686, 0.0664], \
    [0.0563, 0.0406, 0.0414], \
    [0.0365, 0.0626, 0.0278], \
    [0.0531, 0.0787, 0.0315], \
    [0.0126, 0.0166, 0.0220], \
    [0.0855, 0.0502, 0.0418], \
    [0.0500, 0.0321, 0.0279], \
    [0.0372, 0.1239, 0.0528], \
    [0.0196, 0.0242, 0.0226], \
    [0.0548, 0.0353, 0.0966], \
    [0.1116, 0.0603, 0.1003], \
    [0.0679, 0.0578, 0.0472], \
    [0.0263, 0.0152, 0.0215], \
    [0.0391, 0.0318, 0.0537], \
    [0.0244, 0.0775, 0.0192], \
    [0.0492, 0.0737, 0.0543], \
    [0.0433, 0.0636, 0.0743], \
    [0.0153, 0.0113, 0.0187], \
    [0.0353, 0.0292, 0.0485], \
    [0.0603, 0.0469, 0.1315]]
    )

#source does not have emission for X amino acid label
E_delta = (E[:,:-1] - E_source.T).round(decimals=4)
E_delta

array([[-0.008 , -0.046 ,  0.0564, -0.0009,  0.0231, -0.0449,  0.0078,
        -0.0164,  0.0459, -0.0356,  0.0077, -0.0351,  0.0215, -0.0147,
         0.0241,  0.0104, -0.0019, -0.0004,  0.0249, -0.0253],
       [-0.0045, -0.0286, -0.0057,  0.0027,  0.1029, -0.0182,  0.0029,
        -0.0979,  0.0317, -0.0223,  0.003 ,  0.0039,  0.0172,  0.0478,
        -0.0039, -0.0292, -0.004 , -0.0004,  0.016 , -0.0187],
       [-0.0028, -0.0254,  0.02  ,  0.0014,  0.0255,  0.0162,  0.0729,
        -0.0296,  0.0228, -0.0811,  0.0032, -0.0213,  0.0067, -0.0346,
         0.0319, -0.0081, -0.0086, -0.0005,  0.0854, -0.0812]])

### Step 6: Compute HMM Prediction Performance on Train Data
    We can compare against performance of traditional HMM from [5] as sanity check:
        Overall Accuracy: 44.38%
        Helix Accuracy (H): 90.46%
        Beta-Sheet Accuracy (E): 4.56%
        Coil Accuracy (C): 28.05%
        
     *Some slight difference is expected because they were only able to train and test on CB531 whereas we will be training on CB6113 and testing on CB531.
    

In [ ]:
#make predictions
start = time.time()
train_predictions = getPredictions(train_df_formatted, T, E, pi)
train_acc, train_prec, train_rec, train_cnts = HMMaccuracy(train_predictions, q=3)
end = time.time()
print(f"Time predict on training data is approx: {round((end-start),2)} seconds")

Time predict on training data is approx: 25.9 seconds


In [ ]:
#Our HMM performance

print(f"Accuracy: {round(train_acc,4)} \n")
print(f"Precision (H,C,E):\n\t {train_prec.round(decimals = 4)} \n")
print(f"Recall (H,C,E):\n\t {train_rec.round(decimals = 4)} \n")
print("Counts (H,C,E) x (H,C,E):")
print(train_cnts)

Accuracy: 0.4487 

Precision (H,C,E):
	 [0.425  0.6161 0.472 ] 

Recall (H,C,E):
	 [0.9461 0.1838 0.0472] 

Counts (H,C,E) x (H,C,E):
[[379538  16968   4655]
 [329085  75752   7233]
 [184338  30243  10629]]


### Step 7: Compute HMM Performance on Dev Data
    We can compare against performance of traditional HMM from [5] as sanity check:
            Overall Accuracy: 44.38%
            Helix Accuracy (H): 90.46%
            Beta-Sheet Accuracy (E): 4.56%
            Coil Accuracy (C): 28.05%
        
     *Some slight difference is expected because they were only able to train and test on CB531 whereas we will be training on CB6113 and testing on CB531.

In [ ]:
start = time.time()
dev_predictions = getPredictions(dev_df_formatted, T, E, pi)
dev_acc, dev_prec, dev_rec, dev_cnts = HMMaccuracy(dev_predictions, q=3)
end = time.time()
print(f"Time predict on dev data is approx: {round((end-start),2)} seconds")

Time predict on dev data is approx: 2.9 seconds


In [ ]:
print(f"Accuracy: {round(dev_acc,4)} \n")
print(f"Precision (H,C,E):\n\t {dev_prec.round(decimals = 4)} \n")
print(f"Recall (H,C,E):\n\t {dev_rec.round(decimals = 4)} \n")
print("Counts (H,C,E) x (H,C,E):")
print(dev_cnts)

Accuracy: 0.4498 

Precision (H,C,E):
	 [0.423  0.6198 0.4801] 

Recall (H,C,E):
	 [0.9399 0.1989 0.0594] 

Counts (H,C,E) x (H,C,E):
[[41435  1999   651]
 [36090  9206   996]
 [20424  3649  1521]]
